### Install and Import Packages

##### Outline
- Configuration
- Prompts
- Helper Functions
- Execute Process

##### 1. Configuration

In [1]:
###Install Packages

%pip install openai==1.13.3
%pip install bs4
%pip install pandas

StatementMeta(, , , SessionStarting, )

In [ ]:
###Import Libraries

import os
import time
import json
from openai import AzureOpenAI
import pandas as pd 
from bs4 import BeautifulSoup 
from string import Template
from time import sleep
import uuid


StatementMeta(, , , Waiting, )

In [ ]:
#setup environment variables
v_api_version = "2024-02-15-preview"
v_api_key = "[YOUR API KEY]"
v_api_endpoint = "[YOUR ENDPOINT]"


StatementMeta(, , , Waiting, )

##### 2. Prompt Templates

In [ ]:
# Prompt for extracting skills from project experience brief 
prompt_skill_template = """
From the description below give me a comma separated list of of technologies that were used.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses
1. The technology should be the short format and do not include a description.
2. If a technology cannot be determined return the text "Unable to determine technologies"   

Description:
$r_text
"""

StatementMeta(, , , Waiting, )

##### 3. Helper Functions

In [ ]:
# Function to call the OpenAI API
def process_prompt(prompt, system_msg):

    client = AzureOpenAI(
    azure_endpoint= v_api_endpoint,
    api_key= v_api_key,
    api_version= v_api_version,
    )
    v_completion = client.chat.completions.create(
        model="gpt-35-turbo", # easily change the model using whichever model you want
        messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": prompt},
            ],
        #max_tokens=15000,
        #temperature=0,
    )
    v_nlp_results = v_completion.choices[0].message.content
    sleep(8)
    return v_nlp_results

# Strips html from the provided text
def strip_html(input):
    v_sanitized = ""
    try:
        soup = BeautifulSoup(input, 'html.parser')
        v_sanitized = soup.get_text()
    except Exception as e:
        display("Unable to strip html " + e)

    return v_sanitized

# Attempsts to extract skills from the provided data
def extract_skills(obj_Exp, skill_list):
     #get solution summary and work_performed
     v_summary = strip_html(obj_Exp["solution_summary"])
     v_wrk_perf = strip_html(obj_Exp["work_performed"])
     v_sys_msg = "You are a helpful assistant."
     v_promp = Template(prompt_skill_template).substitute(r_text=v_wrk_perf)

     v_skills = []
     v_result = process_prompt(v_promp,v_sys_msg)

     if "," in v_result:
        v_skills = v_result.split(",")
     else: 
        v_skills = [v_result]

    
     return v_skills 

StatementMeta(, , , Waiting, )

##### 4. Execute Process

In [ ]:

#Iterate over the exp records and extract skills/technology
sprk_df = spark.sql("select id, solution_summary, work_performed from lh_neural_ninjas.people_data_project_experience LIMIT 10")

df = sprk_df.toPandas() 
v_final_skill_array = []

for index, row in df.iterrows():
    
    v_skill_array = extract_skills(row, any)

    for skill in v_skill_array:
        v_clean_text = skill.lstrip().rstrip('.')
        v_rowid = row["id"]

        if "determine technologies" not in v_clean_text:
            v_final_skill_array.append([str(uuid.uuid4()),row["id"],v_clean_text])

            json_str = json.dumps(v_final_skill_array)
            display(json_str)

if len(v_final_skill_array) > 0:
    v_final_df = pd.DataFrame(v_final_skill_array, columns=['id','project_experience_id', 'skill'])

    v_sprk_df = spark.createDataFrame(v_final_df)
    v_sprk_df.write.mode("overwrite").format("delta").saveAsTable("people_data_project_skills")
    

    display("Done processing.")

StatementMeta(, , , Waiting, )